In [3]:
import os
import numpy as np
import pandas as pd
import time
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr
import coordTransform


In [4]:
date = '20161101' #TODO
day_begin = '07:00:00' #TODO
day_end = '11:00:00' #TODO
day_begin_name = 700 #TODO
day_end_name = 1100 #TODO
DATE = 1  #TODO
Date = '01' #TODO

ORDER_FILE_PATH_WIN = f'F:/大学/第40期PRP/特征提取/坐标转换后的数据/201611{Date}-{day_begin_name}-{day_end_name}.csv' 
DST_PATH = f'F:/大学/第40期PRP/特征提取/Kepler可视化文件/{date}--{day_begin_name}-{day_end_name}.csv' 
TIME_INTERVAL = 20
SPACE_INTERVAL = 70


In [5]:
df = pd.read_csv('F:/大学/第40期PRP/特征提取/坐标转换后的数据/20161101-700-830.csv')

In [6]:
time1 = f'{date} {day_begin}'
time2 = f'{date} {day_end}'
stamp1 = time.mktime(time.strptime(time1, '%Y%m%d %H:%M:%S'))

# 时间窗划分
df['time_id'] = df.timestamp.apply(lambda x: (x - stamp1)//TIME_INTERVAL)

# 空间网格划分
# 1.计算左边界和上边界，左右-x， 上下-y
left = df['x'].min()
down = df['y'].max()

# 2.生成横向和纵向索引
df['row_id'] = df['y'].apply(lambda y: (y - down)//SPACE_INTERVAL)
df['col_id'] = df['x'].apply(lambda x: (x - left)//SPACE_INTERVAL)

print (f'{DATE} 已生成时空索引')

df = df.dropna()
print (f'{DATE} 已处理空值, 当前数据条数：',len(df))

1 已生成时空索引
1 已处理空值, 当前数据条数： 148080


In [7]:
df

,driver_ID,order_ID,timestamp,lon,lat,x,y,time_id,row_id,col_id
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.100935,30.716003,1.158846e+07,3.595921e+06,204.0,-27.0,97.0
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.100935,30.716003,1.158846e+07,3.595921e+06,204.0,-27.0,97.0
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.100935,30.716003,1.158846e+07,3.595921e+06,205.0,-27.0,97.0
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.100935,30.716003,1.158846e+07,3.595921e+06,205.0,-27.0,97.0
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.100935,30.716003,1.158846e+07,3.595921e+06,205.0,-27.0,97.0
...,...,...,...,...,...,...,...,...,...,...
148075,hbvyvmmc8@Appe.ipwoeotmakabaGnAc,maCznmck50Bmsmcrqyhpwqt8dn4.BxBa,1477960068,104.083535,30.656933,1.158653e+07,3.588275e+06,263.0,-136.0,69.0
148076,hbvyvmmc8@Appe.ipwoeotmakabaGnAc,maCznmck50Bmsmcrqyhpwqt8dn4.BxBa,1477960071,104.083875,30.656773,1.158656e+07,3.588254e+06,263.0,-136.0,70.0
148077,hbvyvmmc8@Appe.ipwoeotmakabaGnAc,maCznmck50Bmsmcrqyhpwqt8dn4.BxBa,1477960074,104.084184,30.656622,1.158660e+07,3.588235e+06,263.0,-137.0,70.0
148078,hbvyvmmc8@Appe.ipwoeotmakabaGnAc,maCznmck50Bmsmcrqyhpwqt8dn4.BxBa,1477960077,104.084414,30.656522,1.158662e+07,3.588222e+06,263.0,-137.0,71.0


In [8]:


# 下面开始时空特征提取

#1. 计算瞬时速度
# 排序：先按司机排，同司机按订单排，同订单再按时间排
print('正在计算瞬时速度')
df = df.sort_values(by = ['driver_ID', 'order_ID', 'timestamp']).reset_index(drop = True)
# 将订单id下移一行，用于判断前后数据是否属于同一订单
df['orderFlag'] = df['order_ID'].shift(1)
df['identi'] = (df['orderFlag'] == df['order_ID']) #一个由boolean构成的列，方便后面所有shift完成了之后再删除分界行
# 将坐标，时间戳下移一行，匹配相应轨迹点
df['x1'] = df['x'].shift(1)
df['y1'] = df['y'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)
# 将不属于同一订单的轨迹点删除
df = df[df['identi'] == True]
# 计算相邻轨迹点之间的距离和相差时间
# 距离采用欧式距离
dist = np.sqrt(np.square(df['x'].values - df['x1'].values) + np.square(df['y'].values - df['y1'].values))
ttime = df['timestamp'].values - df['timestamp1'].values
# 计算速度
df['speed'] = dist/ttime
# 删除临时数据
df = df.drop(columns = ['x1', 'y1', 'orderFlag', 'timestamp1', 'identi'])
print('已生成速度')

# 2.计算瞬时加速度
print('正在计算加速度')
df['speed1'] = df['speed'].shift(1)
df['timestamp1'] = df['timestamp'].shift(1)
df['identi'] = df['order_ID'].shift(1)
df = df[df.identi == df.order_ID]
df['acc'] = (df.speed - df.speed1)/(df.timestamp - df.timestamp1)
df = df.drop(columns = ['speed1', 'timestamp1', 'identi'])
print('已生成加速度')

df = df.reset_index(drop = True)

# 下面计算集体/网格平均特征

# 1. 网格平均速度：先求每辆车在网格中的平均速度，然后求网格中所有个体平均速度的军制
# 基于时空网格和估计id分组
print('正在生成平均网格速度')
orderGrouped = df.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id', 'order_ID'])
# 网格在每个时刻（时间窗）的平均速度
grouped_speed = orderGrouped.speed.mean().reset_index()
grouped_speed = grouped_speed.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id'])
grid_speed = grouped_speed.speed.mean()
# 去除异常值
grid_speed = grid_speed.clip(grid_speed.quantile(0.05), grid_speed.quantile(0.95))
print('已生成网格平均速度')

# 2. 网格平均加速度
print('正在生成网格平均加速度')
gridGrouped = df.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id'])
grid_acc = gridGrouped.acc.mean()
print('已生成网格平均加速度')

# 3.网格浮动车流量
print('正在生成网格浮动车数量')
grouped_volume = orderGrouped.speed.last().reset_index() #每个时空网格中的每个order只保留一辆（用last（）来取）
grouped_volume = grouped_volume.groupby(['lon', 'lat' ,'row_id', 'col_id', 'time_id'])
grid_volume = grouped_volume['speed'].size()
grid_volume = grid_volume.clip(grid_volume.quantile(0.05), grid_volume.quantile(0.95))
print('已生成网格浮动车数量')

# 4.网格车速标准差
print('正在生成网格车速标准差')
grid_v_std = gridGrouped.speed.std(ddof=0)
# 去除异常值
grid_v_std = grid_v_std.clip(grid_v_std.quantile(0.05), grid_v_std.quantile(0.95))
print('已生成网格车速标准差')

# 5.网格平均停车次数
print('正在生成网格平均停车次数')
stopNum = gridGrouped.speed.agg(lambda x: (x==0).sum())
grid_stop = pd.concat((stopNum, grid_volume), axis = 1)
grid_stop['stopNum'] = stopNum.values/ grid_volume.values
grid_stop = grid_stop['stopNum']
grid_stop = grid_stop.clip(0, grid_stop.quantile(0.95))
print('已生成网格平均停车次数')

feature = pd.concat([grid_speed, grid_acc, grid_volume, grid_v_std, grid_stop], axis = 1).reset_index()
feature.columns = ['lon', 'lat' ,'row_id','col_id', 'time_id', 'aveSpeed', 'gridAcc', 'volume', 'speedStd', 'stopNum']
print('已整理网格特征')
feature.sort_values(['stopNum']).reset_index(drop=True)
feature['date'] = DATE

正在计算瞬时速度
已生成速度
正在计算加速度
已生成加速度
正在生成平均网格速度
已生成网格平均速度
正在生成网格平均加速度
已生成网格平均加速度
正在生成网格浮动车数量
已生成网格浮动车数量
正在生成网格车速标准差
已生成网格车速标准差
正在生成网格平均停车次数


KeyboardInterrupt: 

In [ ]:
feature.to_csv(DST_PATH, index=False)